In [ ]:
# Importing the requisite libraries 

import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import torch
import torchvision

from torchvision import transforms 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

DIR_INPUT = '/kaggle/input/global-wheat-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [ ]:
train_df = pd.read_csv('/kaggle/input/global-wheat-detection/train.csv')
submit = pd.read_csv("/kaggle/input/global-wheat-detection/sample_submission.csv")
train_df.shape

In [ ]:
train_df.head(5)

In [ ]:
# Dropping unrequired columns
train_df = train_df.drop(['width', 'height', 'source'], axis=1)

In [ ]:
# Checking the number of unique images in train_df
train_df['image_id'].nunique() 

In [ ]:
# Checking the maximum number of bounding boxes present in an image
(train_df['image_id'].value_counts()).max()  

In [ ]:
# Checking the minimum number of bounding boxes present in an image
(train_df['image_id'].value_counts()).min()  

In [ ]:
# bbox in each row is formatted [xmin, ymin, width, height]
# Splitting the bounding box  

train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)
train_df['x'] = train_df['x'].astype(np.float)
train_df['y'] = train_df['y'].astype(np.float)
train_df['w'] = train_df['w'].astype(np.float)
train_df['h'] = train_df['h'].astype(np.float)

In [ ]:
train_df.head()

In [ ]:
# Splitting the data into test and validation sets

image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]

valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

In [ ]:
train_df.shape, valid_df.shape

In [ ]:
# Converting the numpy image to tensor
trans = transforms.Compose([transforms.ToTensor()])   

In [ ]:
# Creating a custom dataset

class WheatDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None,train=True):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.train=train

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        #Applying transformations
        if self.transforms is not None:  
            image = self.transforms(image)
         
         # For test data
        if(self.train==False): 
            return image, image_id
        
        # For train and val data        
           
        # getting all the records for a particular image_id
        records = self.df[self.df['image_id'] == image_id]   
        
     
        # initially boxes = [xmin, ymin, width, height]
        boxes = records[['x', 'y', 'w', 'h']].values
        # converting width to xmax
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        # converting height to ymax
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
       
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # Now, boxes = [xmin, ymin, xmax, ymax]
        # area = w*h => (ymax-ymin)*(xmax-xmin) 
        # creating a series of area
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        # Since there is only 1 class present (wheat) in all the bboxes
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)   

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        return image, target, image_id 

In [ ]:
train_dir = '/kaggle/input/global-wheat-detection/train'
test_dir = '/kaggle/input/global-wheat-detection/test'

In [ ]:
# Returns the average loss
class Averager:       
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1
        
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations     

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
# collate function is being used because in one batch, we are passing 16 images, but the number of bbox in each of the 16 images may be different. To allow passing tensors of different sizes, collate function is used.        
# * unpacks the iterable
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = WheatDataset(train_df, train_dir, trans, True)
valid_dataset = WheatDataset(valid_df, train_dir, trans, True)


train_data_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# images, targets, image_ids because Dataset class was returning these
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

boxes = targets[4]['boxes'].cpu().numpy().astype(np.int32)
sample = images[4].permute(1,2,0).cpu().numpy().copy()  #Without using .copy(), there will be an error
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    #cv2.rectangle(image, start_point, end_point, color, thickness)
    cv2.rectangle(sample,
                  (int(box[0]), int(box[1])),
                  (int(box[2]), int(box[3])),
                  (220, 0, 0),3)
    
# ax.set_axis_off()
ax.imshow(sample)

In [ ]:
# loading the model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
# 1 class is for wheat and the other is the background
num_classes = 2  

# Getting the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

WEIGHTS_FILE = '../input/fasterrcnn/fasterrcnn_resnet50_fpn_best.pth'

# loading the pre-trained weights
model.load_state_dict(torch.load(WEIGHTS_FILE))  


In [ ]:
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

num_epochs = 5

loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)   ##Return the loss

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)  #Average out the loss

        optimizer.zero_grad()
        losses.backward()
        # updating the parameters
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # updating the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")

In [ ]:
test_dataset = WheatDataset(submit,test_dir,trans,False)

In [ ]:
test_data_loader = DataLoader( test_dataset, batch_size=8, shuffle=False)

In [ ]:
detection_threshold = 0.45

In [ ]:
# Defining the format for storing prediction results
def format_prediction_string(boxes, scores): 
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
#Making the predictions
results=[]
model.eval()
 
for images, image_ids in test_data_loader:
    
    images = list(image.to(device) for image in images)
    outputs = model(images)
    
    for i, image in enumerate(images):
        boxes = outputs[i]['boxes'].data.cpu().numpy()  #[Xmin,Ymin,Xmax,Ymax]
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        
        image_id = image_ids[i]
        
        #converting the box format to [Xmin,Ymin,W,H]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]         
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1] 
        
        #Storing the image id and boxes and scores in result dict
        
        result = {                                     
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)}
          
        #Appending the result dict to results list
        results.append(result)  

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

In [ ]:
sample = images[1].permute(1,2,0).cpu().numpy().copy()
boxes = outputs[1]['boxes'].data.cpu().numpy()
scores = outputs[1]['scores'].data.cpu().numpy()

boxes = boxes[scores >= detection_threshold].astype(np.int32)

In [ ]:
# plotting some of the predictions

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (int(box[0]), int(box[1])),
                  (int(box[2]), int(box[3])),
                  (220, 0, 0), 2)
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
test_df.to_csv('submission.csv', index=False)
